#Develop clustering code

In [108]:
%matplotlib inline
import pyfits, glob, sys, pickle
from skymaps import SkyDir
os.chdir(os.path.expandvars('$FERMI/skymodels/P301_6years/uw970'))
from uw.like2.pub import healpix_map; reload(healpix_map)
from uw.like2.analyze import hptables; reload(hptables)
from uw.like2.pipeline import check_ts; reload(check_ts)

<module 'uw.like2.pipeline.check_ts' from '/afs/slac/g/glast/groups/catalog/pointlike/python/uw/like2/pipeline/check_ts.py'>

###Organize data from the TS table

In [105]:
tables = 'ts_kde'
mints=10
table_filename = 'hptables_%s_512.fits' % tables
if not os.path.exists(table_filename):
    healpix_map.assemble_tables(tables.split('_'))
tsdata = check_ts.TSdata('.',table_filename, 'ts')
indices = tsdata.indices(mints, 0)
print 'found %d pixels with TS>%d' % (len(indices), mints)

found 6459 pixels with TS>10


###Make clusters

In [96]:
def cluster(indices, quiet=False):
    if not quiet:
        print 'Clustering %d pixels...' % len(indices)
        sys.stdout.flush()
    ret = []
    rem = indices
    while len(rem)>0:
        clu, rem = check_ts.grow(rem)
        ret.append(clu)
    return ret


In [97]:
%time clusters = cluster(indices)

Clustering 6459 pixels...
CPU times: user 10min 20s, sys: 7.98 s, total: 10min 28s
Wall time: 10min 44s


###Save or load the cluster list

In [99]:
pickle_file='ts_clusters.pickle'
if os.path.exists(pickle_file):
    print 'loading file %s' % pickle_file
    clusters =pickle.load(open(pickle_file))
else:                      
    pickle.dump(clusters, open(pickle_file, 'w'))

In [100]:
clu = clusters[large_indices[0]]; print len(clu)
len(subclusters(clu, 10))

1


1

In [47]:
for i in large_indices:
    clu = clusters[i]
    ts = cl.group(clu)
    #print i, len(clu), ts
    if ts<30: continue
    subs = subclusters(clu, 25)
    print len(clu), len(subs)

Clustering 1 pixels...
34 1
Clustering 9 pixels...
27 1
Clustering 5 pixels...
38 1
Clustering 11 pixels...
103 1
Clustering 4 pixels...
34 1
Clustering 165 pixels...
298 3
Clustering 82 pixels...
188 3
Clustering 8 pixels...
34 1
Clustering 87 pixels...
298 3
Clustering 121 pixels...
317 3
Clustering 133 pixels...
293 4
Clustering 63 pixels...
227 8
Clustering 14 pixels...
73 1
Clustering 103 pixels...
320 11
Clustering 7 pixels...
71 1
Clustering 8 pixels...
38 2
Clustering 23 pixels...
72 1
Clustering 11 pixels...
47 1
Clustering 6 pixels...
47 1
Clustering 12 pixels...
95 5
Clustering 20 pixels...
86 2
Clustering 8 pixels...
44 1
Clustering 22 pixels...
45 1
Clustering 52 pixels...
168 2
Clustering 61 pixels...
139 3
Clustering 3 pixels...
29 1
Clustering 4 pixels...
30 2
Clustering 343 pixels...
717 3
Clustering 6 pixels...
32 2
Clustering 17 pixels...
60 1
Clustering 4 pixels...
29 1
Clustering 15 pixels...
27 1
Clustering 12 pixels...
110 3
Clustering 10 pixels...
33 3
Clusterin

In [5]:
i = list(sizes).index(max(sizes))
bigone = clusters[i]

###Analyze the cluster

In [70]:
class Cluster(object):
    def __init__(self, rts):
        self.rts = rts
    
    def group(self, clu):
        """ clu: list of pixel indices"""
        # find the position of the maximum
        ts = np.array([self.rts[i] for i in clu])
        self.ts=maxts=max(ts)
        j = np.arange(len(ts))[ts==maxts][0]
        i = clu[j]
        self.sdir = check_ts.sdir(i)
        return self.ts
        
cl = Cluster(tsdata.rts)
cl.group(bigone)
cl.sdir, cl.sdir.l(), cl.sdir.b(), cl.ts, len(bigone)

(SkyDir(97.093,5.305), 205.57617187499997, -2.686724185691604, 104.06334, 717)

In [7]:
# make a DataFrame for the cluster
rts = tsdata.rts
dd =dict([(i, dict(ts=rts[i])) for i in bigone])
df =pd.DataFrame(dd).T.sort_index(by='ts',ascending=False)
df['peak'] = [peak(i) for i in df.index]
sdirs = [check_ts.sdir(i) for i in df.index]
df['ra'] = [s.ra() for s in sdirs]
df['dec']= [s.dec() for s in sdirs]

In [81]:
df.sort_index(by='peak', ascending=False)

,ts,peak,ra,dec
1646737,104.063339,28.662549,97.093175,5.304582
1648786,95.702248,19.929324,97.067488,5.192192
1644689,89.157364,19.897333,97.118857,5.416974
1663125,45.105213,19.788410,96.887478,4.405576
1652883,78.314751,18.317366,97.016096,4.967425
1632413,50.921719,18.008855,98.492875,3.753633
1638555,48.216057,15.872084,98.172084,3.884527
1654931,64.137131,15.490956,96.990389,4.855047
1620125,55.762852,15.201361,98.891561,3.959463
1648789,66.827049,15.125195,97.311973,4.725159


In [8]:
from pointlike import IntVector
def peak(i):
    iv = IntVector()
    nb = check_ts.band.findNeighbors(int(i), iv)
    tsvals = [rts[j] for j in iv] 
    return rts[i] - sum(tsvals)/len(tsvals)
peak(df.index[0])

28.662549495697021

##Try sub-clusters
First break up with TS=25 threshold

In [101]:
def subclusters(clust, mints=10, quiet=True):
    cut_cluster = filter(lambda i: rts[i]>mints, clust)
    if len(cut_cluster)==0: return []
    clusters = cluster(cut_cluster, quiet)
    return clusters

def properties(cluster):
    return (len(cluster), max([rts[i] for i in cluster]))
def ts_max(cluster):
    return max([rts[i] for i in cluster])

In [102]:
sizes = np.array([len(c) for c in clusters])

large_indices = np.arange(len(clusters))[sizes>25]

maxtsvals = np.array([cl.group(clusters[i]) for i in large_indices])
sum(sizes>25), sum(maxtsvals>25)

(34, 32)

In [103]:
def split_clusters(clusters, maxsize=25, split_ts = 25):
    # loop over clusters, make a list of split clusters
    splits = []
    for clu in clusters:
        if len(clu)<maxsize: continue
        t = subclusters(clu, split_ts)
        if len(t)<2: continue
        splits += t
    return splits
len(split_clusters(clusters))

49

##Final test 

In [104]:
pwd

u'/nfs/farm/g/glast/g/catalog/pointlike/skymodels/P301_6years/uw970'

In [109]:
check_ts.make_seeds('', table_filename, seedroot='T970')

Clustering 6459 pixels...
Found 1946 clusters
Found 1946 clusters
Added split clusters, now 1993 total


1993